In [1]:
import pandas as pd
import numpy as np

from datetime import date
from datetime import datetime
import datetime

import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

datos = pd.read_csv('C:/Users/Paula/Desktop/TFG/rtve-observations-episodes-2019-11-01.zip')

In [2]:
datos.head()

,id,date,metric,value,user,referer,os_family,os_version,browser_family,browser_version,...,podcast_permalink,podcast_topic,entries_processed,hostname,city,region,country,location,zip_code,time_zone
0,1a50eda4-26b1-47df-949d-4827665b79e3,2019-11-01T00:56:36.000,Número de descargas de tipo 206,1,6624207285322504776,NaN,Android,6.0.1,Android,6.0.1,...,RADIO-NACIONAL,Informativos,4,NaN,Palencia,Castilla y León,España,"42.0021, -4.5237",34004,Europe/Madrid
1,da19819a-3784-41ef-9ecd-67d6e4be37aa,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1,1093782487219863411,NaN,Android,8.0.0,Android,8.0.0,...,RADIO-NACIONAL,Informativos,21,NaN,Madrid,Madrid,España,"40.4143, -3.7016",28021,Europe/Madrid
2,a60f41aa-6b3d-4fe9-bf9b-6311c57abfc1,2019-11-01T00:49:33.000,Número de descargas de tipo 206,1,-3042089131472286197,RTVE,Windows,7,Chrome,77.0.3865,...,RADIO-CLASICA,Música,7,NaN,San Sebastián Tutla,Oaxaca,México,"17.0645, -96.6705",71242,America/Mexico_City
3,e85a8032-abad-48b4-9bd2-2765a4b730f4,2019-11-01T01:27:12.000,Número de descargas de tipo 206,1,-5995978442346962145,NaN,Android,9,Other,NaN,...,RADIO-NACIONAL,Entretenimiento,15,NaN,La Matanza de Acentejo,Canarias,España,"28.4524, -16.4472",38370,Atlantic/Canary
4,f3dabef8-f024-4f49-a04e-db6bbc1d5e2b,2019-11-01T00:56:50.000,Petición de un posible BOT,1,-233953730499517940,RTVE,Android,7.0,Chrome Mobile,76.0.3809,...,RADIO-3,Música,1,NaN,NaN,NaN,NaN,"90.0000, 0.0000",NaN,NaN


In [3]:
#Se eliminan las columnas con los datos que no son necesarios.
Datos = datos.drop(['id', 'value', 'os_version', 'browser_version', 'device_brand', 'device_model', 'episode_file_size',  'entries_processed', 'hostname', 'location', 'zip_code'], axis=1)

In [4]:
Datos.head()

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid
2,2019-11-01T00:49:33.000,Número de descargas de tipo 206,-3042089131472286197,RTVE,Windows,Chrome,Dispositivo no registrado,1572524596000.mp3,A través de un espejo - 31/10/19,3513239,A través de un espejo,RADIO-CLASICA,Música,San Sebastián Tutla,Oaxaca,México,America/Mexico_City
3,2019-11-01T01:27:12.000,Número de descargas de tipo 206,-5995978442346962145,NaN,Android,Other,Smartphone,1572175203168.mp3,Encuentros con la vida después de la vida,3201395,Espacio en blanco,RADIO-NACIONAL,Entretenimiento,La Matanza de Acentejo,Canarias,España,Atlantic/Canary
4,2019-11-01T00:56:50.000,Petición de un posible BOT,-233953730499517940,RTVE,Android,Chrome Mobile,Dispositivo no registrado,1571416123127.mp3,Sueño con la tolerancia (1ª parte),3536124,Videodrome,RADIO-3,Música,NaN,NaN,NaN,NaN


In [5]:
Datos.shape

(10000, 17)

Las categorías recogidas en la columna 'metric' son las siguientes:

In [6]:
Datos['metric'].value_counts(dropna=False)

Número de descargas de tipo 206              3841
Petición de tipo HEAD                        3216
Número de descargas parciales de tipo 206    1262
Número de descargas parciales de tipo 200    1103
Petición no 200 o 206                         340
Número de descargas de tipo 200               204
Petición de un recurso no MP3                  17
Petición de un posible BOT                     17
Name: metric, dtype: int64

In [7]:
#Se eliminan los datos que no sean de tipo 206, 200 o descargas parciales.No son interesantes para el análisis.
Datos = Datos[Datos.metric.isin(['Número de descargas de tipo 206', 'Número de descargas parciales de tipo 206', 'Número de descargas parciales de tipo 200','Número de descargas de tipo 200' ])]

In [8]:
Datos.metric

0                 Número de descargas de tipo 206
1       Número de descargas parciales de tipo 200
2                 Número de descargas de tipo 206
3                 Número de descargas de tipo 206
5       Número de descargas parciales de tipo 206
                          ...                    
9994              Número de descargas de tipo 206
9995    Número de descargas parciales de tipo 206
9997    Número de descargas parciales de tipo 200
9998              Número de descargas de tipo 206
9999              Número de descargas de tipo 206
Name: metric, Length: 6410, dtype: object

In [9]:
Datos['metric'].value_counts(dropna=False)

Número de descargas de tipo 206              3841
Número de descargas parciales de tipo 206    1262
Número de descargas parciales de tipo 200    1103
Número de descargas de tipo 200               204
Name: metric, dtype: int64

In [10]:
#Se eliminan los datos cuya duración del episodio es 0.
Datos= Datos[Datos.episode_duration!=0]

In [11]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid


Para el sistema de recomendación usuario-usuario se necesitan las siguientes columnas:

    usuarios
    podcast_title
    metric

In [12]:
matriz_user = Datos[['user', 'podcast_title',  'metric']]

In [13]:
matriz_user

,user,podcast_title,metric
0,6624207285322504776,24 horas,Número de descargas de tipo 206
1,1093782487219863411,24 horas,Número de descargas parciales de tipo 200
2,-3042089131472286197,A través de un espejo,Número de descargas de tipo 206
3,-5995978442346962145,Espacio en blanco,Número de descargas de tipo 206
5,-9171307944933687357,Espacio en blanco,Número de descargas parciales de tipo 206
...,...,...,...
9993,3874912413170459905,Música y significado,Número de descargas parciales de tipo 200
9994,413781770287457576,Sonideros,Número de descargas de tipo 206
9995,-3304109411418471470,Las mañanas de RNE con Pepa Fernández,Número de descargas parciales de tipo 206
9997,-1986203695208531041,Las mañanas de RNE con Pepa Fernández,Número de descargas parciales de tipo 200


In [14]:
#Se muestran el número de registros de datos para cada episodio.
matriz_user['podcast_title'].value_counts()

Las mañanas de RNE con Pepa Fernández    533
Espacio en blanco                        526
Las mañanas de RNE con Íñigo Alfonso     243
Hoy empieza todo con Ángel Carmona       210
180 grados                               206
                                        ... 
Rebobinando                                1
La ciencia en la alcoba                    1
Audios para recordar                       1
La Belle Epoque y las Vanguardias          1
Asuntos propios                            1
Name: podcast_title, Length: 365, dtype: int64

In [15]:
#Se usa el cuantil 0.95 para eliminar los datos que no son relevantes para nuestro estudio.
matriz_user['podcast_title'].value_counts().quantile(0.95)

64.60000000000002

In [16]:
#Se muestran los datos cuyo número de reproducciones de cada podcast sea superior a 64
limite = 64 
mv_podcast_title = matriz_user['podcast_title'].value_counts().drop(matriz_user['podcast_title'].value_counts()[matriz_user['podcast_title'].value_counts()<=limite].index)

In [17]:
mv_podcast_title

Las mañanas de RNE con Pepa Fernández        533
Espacio en blanco                            526
Las mañanas de RNE con Íñigo Alfonso         243
Hoy empieza todo con Ángel Carmona           210
180 grados                                   206
24 horas                                     180
Documentos RNE                               173
El sótano                                    163
Cuando los elefantes sueñan con la música    135
Siglo 21                                     135
Turbo 3                                      109
No es un día cualquiera                      102
Gente despierta                               96
Saltamontes                                   92
Sólo jazz                                     86
Boletines RNE                                 85
Música antigua                                71
Música a la carta                             67
Sinfonía de la mañana                         65
Name: podcast_title, dtype: int64

In [18]:
mv_podcast_title.count()

19

In [19]:
matriz_user['podcast_title'].value_counts().count()

365

In [20]:
mv_podcast_title.index

Index(['Las mañanas de RNE con Pepa Fernández', 'Espacio en blanco',
       'Las mañanas de RNE con Íñigo Alfonso',
       'Hoy empieza todo con Ángel Carmona', '180 grados', '24 horas',
       'Documentos RNE', 'El sótano',
       'Cuando los elefantes sueñan con la música', 'Siglo 21', 'Turbo 3',
       'No es un día cualquiera', 'Gente despierta', 'Saltamontes',
       'Sólo jazz', 'Boletines RNE', 'Música antigua', 'Música a la carta',
       'Sinfonía de la mañana'],
      dtype='object')

In [21]:
#Nueva matriz con los títulos de podcast más relevantes.
matriz_user= matriz_user[matriz_user['podcast_title'].isin(mv_podcast_title.index)]

In [22]:
matriz_user

,user,podcast_title,metric
0,6624207285322504776,24 horas,Número de descargas de tipo 206
1,1093782487219863411,24 horas,Número de descargas parciales de tipo 200
3,-5995978442346962145,Espacio en blanco,Número de descargas de tipo 206
5,-9171307944933687357,Espacio en blanco,Número de descargas parciales de tipo 206
6,180705575781551235,Cuando los elefantes sueñan con la música,Número de descargas parciales de tipo 206
...,...,...,...
9983,1272847932212292212,Documentos RNE,Número de descargas de tipo 206
9986,1143741598281359418,Espacio en blanco,Número de descargas parciales de tipo 200
9989,-1673116384655516914,Siglo 21,Número de descargas de tipo 206
9995,-3304109411418471470,Las mañanas de RNE con Pepa Fernández,Número de descargas parciales de tipo 206


In [23]:
#Se muestra la lista con el número de registros para cada usuario de la matriz.
matriz_user['user'].value_counts()

 3413815924711635741    6
 4309534277872859934    5
-6392471075757312178    5
-3899160707308511105    5
 2319800527054000783    5
                       ..
 5326311068826200048    1
-2508775471503505587    1
-7687816708156734469    1
-3886409423336739881    1
 5733484840755179275    1
Name: user, Length: 2830, dtype: int64

In [24]:
#De la misma forma que anteriormente, se calcula el cuantil 0.95 para el registro de usuarios.
matriz_user['user'].value_counts().quantile(0.95)

2.0

In [25]:
matriz_user['user'].value_counts()[matriz_user['user'].value_counts()==2]

-294981485933620270     2
 3263381398100302397    2
 6560946998274433395    2
 73191902376357969      2
-7916316410268307462    2
                       ..
 8496864328721366403    2
 8202147433566467953    2
 6462726320969565949    2
-4950925412519875905    2
-3665857937421428519    2
Name: user, Length: 278, dtype: int64

In [26]:
limite_usuario = 2
mv_user = matriz_user['user'].value_counts().drop(matriz_user['user'].value_counts()[matriz_user['user'].value_counts()<limite_usuario].index)

In [27]:
mv_user

 3413815924711635741    6
 4309534277872859934    5
-6392471075757312178    5
-3899160707308511105    5
 2319800527054000783    5
                       ..
 8496864328721366403    2
 8202147433566467953    2
 6462726320969565949    2
-4950925412519875905    2
-3665857937421428519    2
Name: user, Length: 347, dtype: int64

In [28]:
#Se rehace la matriz con eliminando los usuarios no representativos.
mv_user.index

Int64Index([ 3413815924711635741,  4309534277872859934, -6392471075757312178,
            -3899160707308511105,  2319800527054000783, -8923064467251557676,
            -6244236089212606076, -2210243496593326803, -2899978134859644175,
            -5346441331584591926,
            ...
            -7426848479575804766, -2450467939204667190,  -659757798245483536,
            -1986203695208531041,  1676508636246338393,  8496864328721366403,
             8202147433566467953,  6462726320969565949, -4950925412519875905,
            -3665857937421428519],
           dtype='int64', length=347)

In [29]:
matriz_user=matriz_user[matriz_user['user'].isin(mv_user.index)]

In [30]:
matriz_user

,user,podcast_title,metric
5,-9171307944933687357,Espacio en blanco,Número de descargas parciales de tipo 206
6,180705575781551235,Cuando los elefantes sueñan con la música,Número de descargas parciales de tipo 206
13,8172620313194145685,24 horas,Número de descargas parciales de tipo 206
23,8547547893280042324,Espacio en blanco,Número de descargas de tipo 206
30,-7829275662668923504,Documentos RNE,Número de descargas de tipo 206
...,...,...,...
9935,7741827251431184522,Las mañanas de RNE con Pepa Fernández,Número de descargas de tipo 206
9956,-977354010742103617,Espacio en blanco,Número de descargas de tipo 206
9962,3192517273246231338,180 grados,Número de descargas parciales de tipo 200
9977,-3665857937421428519,Turbo 3,Número de descargas de tipo 206


In [31]:
matriz_user[matriz_user['user']==3413815924711635741]

,user,podcast_title,metric
423,3413815924711635741,24 horas,Número de descargas de tipo 200
705,3413815924711635741,24 horas,Número de descargas de tipo 200
5681,3413815924711635741,24 horas,Número de descargas de tipo 206
5941,3413815924711635741,24 horas,Número de descargas de tipo 200
6411,3413815924711635741,24 horas,Número de descargas de tipo 200
7488,3413815924711635741,24 horas,Número de descargas de tipo 200


In [32]:
#Ejemplo de recuento de veces que cada usuario ha escuchado el podcast '24 horas'
matriz_user[matriz_user.podcast_title=='24 horas'].groupby('user').count()

,podcast_title,metric
user,,
-8971042844030754723,2,2
-8923064467251557676,5,5
-6782208137328013812,2,2
-5925185636683349085,4,4
-5497853561906093199,2,2
-4760526543150040294,2,2
-2727393755660254820,2,2
-1518865233211124196,3,3
-1197079027062428462,2,2


Para el sistema de recomendación a implementar, los datos necesarios son 'user' y 'podcast_title'

In [33]:
 matrix= matriz_user[['user', 'podcast_title']]

In [34]:
matrix

,user,podcast_title
5,-9171307944933687357,Espacio en blanco
6,180705575781551235,Cuando los elefantes sueñan con la música
13,8172620313194145685,24 horas
23,8547547893280042324,Espacio en blanco
30,-7829275662668923504,Documentos RNE
...,...,...
9935,7741827251431184522,Las mañanas de RNE con Pepa Fernández
9956,-977354010742103617,Espacio en blanco
9962,3192517273246231338,180 grados
9977,-3665857937421428519,Turbo 3


In [35]:
mat_fin= matrix.groupby('user').count()

In [36]:
mat_fin

,podcast_title
user,
-9171307944933687357,2
-9153667398542181736,2
-9072037894646033789,2
-9040188660338137877,2
-8971042844030754723,2
...,...
8844490510404225002,3
8969517547138839030,3
9037917513063532329,2


In [37]:
#Función para definir la matriz de interacciones entre usuario y podcast.
#Los valores representan el número de veces que cada usuario ha escuchado un podcast_concreto.
for k in mv_podcast_title.index:
    
     mat_fin[k]= matrix[matrix.podcast_title==k].groupby('user').count()
        


In [38]:
mat_fin.fillna(0)

,podcast_title,Las mañanas de RNE con Pepa Fernández,Espacio en blanco,Las mañanas de RNE con Íñigo Alfonso,Hoy empieza todo con Ángel Carmona,180 grados,24 horas,Documentos RNE,El sótano,Cuando los elefantes sueñan con la música,Siglo 21,Turbo 3,No es un día cualquiera,Gente despierta,Saltamontes,Sólo jazz,Boletines RNE,Música antigua,Música a la carta,Sinfonía de la mañana
user,,,,,,,,,,,,,,,,,,,,
-9171307944933687357,2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9153667398542181736,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9072037894646033789,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9040188660338137877,2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8971042844030754723,2,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8844490510404225002,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8969517547138839030,3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9037917513063532329,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
mat_final= mat_fin.drop(columns='podcast_title').fillna(0)

In [40]:
mat_final

,Las mañanas de RNE con Pepa Fernández,Espacio en blanco,Las mañanas de RNE con Íñigo Alfonso,Hoy empieza todo con Ángel Carmona,180 grados,24 horas,Documentos RNE,El sótano,Cuando los elefantes sueñan con la música,Siglo 21,Turbo 3,No es un día cualquiera,Gente despierta,Saltamontes,Sólo jazz,Boletines RNE,Música antigua,Música a la carta,Sinfonía de la mañana
user,,,,,,,,,,,,,,,,,,,
-9171307944933687357,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9153667398542181736,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9072037894646033789,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9040188660338137877,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8971042844030754723,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8844490510404225002,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8969517547138839030,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9037917513063532329,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
#Se calcula la densidad de la matriz, es decir, la cantidad de datos no nulos que tenemos en porcentaje.
valores = mat_final.values
density = float(len(valores.nonzero()[0]))
density /= (valores.shape[0] * valores.shape[1])
density *= 100
print('Density: {:4.2f}%'.format(density))



Density: 6.32%


In [42]:
valores

array([[0., 2., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 2.]])

Algo más de un 6% de densidad indica que casi el 94% de la matriz está completada con ceros.


In [43]:
user_index= mat_final.index.astype(object)

In [44]:
mat_final=mat_final.reindex(user_index)

In [45]:
mat_final.index

Index([-9171307944933687357, -9153667398542181736, -9072037894646033789,
       -9040188660338137877, -8971042844030754723, -8923064467251557676,
       -8889811117182609516, -8734834306713571602, -8699370630136005426,
       -8673793366812442585,
       ...
        8641166364776784164,  8763933379146723721,  8777099265100368196,
        8794478463476106182,  8827030174391708021,  8844490510404225002,
        8969517547138839030,  9037917513063532329,  9061649906012421828,
        9092723879535892351],
      dtype='object', name='user', length=347)

**Breve ejemplo de cómo se calcula la distancia euclídea entre dos puntos (usuarios).**

In [46]:
user1= mat_final.loc[-8923064467251557676]
user2= mat_final.loc[-8734834306713571602]
user3= mat_final.loc[-8673793366812442585]

In [47]:
user1

Las mañanas de RNE con Pepa Fernández        0.0
Espacio en blanco                            0.0
Las mañanas de RNE con Íñigo Alfonso         0.0
Hoy empieza todo con Ángel Carmona           0.0
180 grados                                   0.0
24 horas                                     5.0
Documentos RNE                               0.0
El sótano                                    0.0
Cuando los elefantes sueñan con la música    0.0
Siglo 21                                     0.0
Turbo 3                                      0.0
No es un día cualquiera                      0.0
Gente despierta                              0.0
Saltamontes                                  0.0
Sólo jazz                                    0.0
Boletines RNE                                0.0
Música antigua                               0.0
Música a la carta                            0.0
Sinfonía de la mañana                        0.0
Name: -8923064467251557676, dtype: float64

In [48]:
user2

Las mañanas de RNE con Pepa Fernández        0.0
Espacio en blanco                            0.0
Las mañanas de RNE con Íñigo Alfonso         0.0
Hoy empieza todo con Ángel Carmona           0.0
180 grados                                   0.0
24 horas                                     0.0
Documentos RNE                               0.0
El sótano                                    0.0
Cuando los elefantes sueñan con la música    0.0
Siglo 21                                     0.0
Turbo 3                                      0.0
No es un día cualquiera                      0.0
Gente despierta                              0.0
Saltamontes                                  0.0
Sólo jazz                                    0.0
Boletines RNE                                0.0
Música antigua                               0.0
Música a la carta                            3.0
Sinfonía de la mañana                        0.0
Name: -8734834306713571602, dtype: float64

In [49]:
#Función que calcula la distancia euclídea entre dos usuarios.
def distancia_user(user1,user2):
  distancia = euclidean(user1,user2)
  return distancia

In [50]:
print("Distancia entre user1 y user2:")
print(distancia_user(user1,user2))

print("Distancia entre user2 y user3:")
print(distancia_user(user2,user3))

print("Distancia entre user1 y user1:")
print(distancia_user(user1,user1))


Distancia entre user1 y user2:
5.830951894845301
Distancia entre user2 y user3:
5.0
Distancia entre user1 y user1:
0.0


In [51]:
#Ejemplo de función itertuples.
def most_similar (user):
    usuario =mat_final.loc[user]
    
    for row in mat_final.itertuples():
        
        
        print(row)

In [52]:
#Se define una función que devuelva el usuario más cercano, a la distancia al mismo.
def mas_cercano (user):
  usuario = mat_final.loc[user]
  menor_distancia = float('inf')
  
  for row in mat_final.itertuples():
    if row.Index==user:
      #no comparar a un usuario consigo mismo
      continue
    distancia_otros = distancia_user(usuario,mat_final.loc[row.Index])
    
   #Se establece que la distancia sea mayor que 1 ya que 0 y 1 proporcionan resultados iguales a los del usuario que comparamos.
    if (distancia_otros < menor_distancia) & (distancia_otros>1) :
       
      #Guardamos esa distancia como la menor posible
      menor_distancia = distancia_otros
      usuario_cercano = row.Index
  return usuario_cercano, menor_distancia
    

In [53]:
mas_cercano (-2132359551882711171)

(-8051681681554922553, 2.23606797749979)

In [54]:
mas_cercano (8507792996028097546)

(-1914691646047876407, 1.4142135623730951)

In [54]:
mas_cercano (-8673793366812442585)

(-9153667398542181736, 2.0)

In [55]:
#Ejemplo para dos usuarios propuestos como más cercanos
mat_final.loc[[-2132359551882711171, -8051681681554922553]]

,Las mañanas de RNE con Pepa Fernández,Espacio en blanco,Las mañanas de RNE con Íñigo Alfonso,Hoy empieza todo con Ángel Carmona,180 grados,24 horas,Documentos RNE,El sótano,Cuando los elefantes sueñan con la música,Siglo 21,Turbo 3,No es un día cualquiera,Gente despierta,Saltamontes,Sólo jazz,Boletines RNE,Música antigua,Música a la carta,Sinfonía de la mañana
user,,,,,,,,,,,,,,,,,,,
-2132359551882711171,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8051681681554922553,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
 mat_final.loc[[8507792996028097546, -1914691646047876407]]

,Las mañanas de RNE con Pepa Fernández,Espacio en blanco,Las mañanas de RNE con Íñigo Alfonso,Hoy empieza todo con Ángel Carmona,180 grados,24 horas,Documentos RNE,El sótano,Cuando los elefantes sueñan con la música,Siglo 21,Turbo 3,No es un día cualquiera,Gente despierta,Saltamontes,Sólo jazz,Boletines RNE,Música antigua,Música a la carta,Sinfonía de la mañana
user,,,,,,,,,,,,,,,,,,,
8507792996028097546,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1914691646047876407,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
